# Imports and load data

In [4]:
from dotenv import load_dotenv
from rag_utils import setup_rag_embeddings, build_rag_chain
from ragas import SingleTurnSample, EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.metrics import Faithfulness, FactualCorrectness
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from langchain_openai import OpenAIEmbeddings

import asyncio
import pandas as pd
import os
import json

load_dotenv(override=True)

model_name = "gpt-4o-mini"
model_name_SAP = "gpt-4o"
DATA_DIR = "cited_papers/"
DATASET = "evaluation_references.json"
proxy_client = get_proxy_client('gen-ai-hub')
llm_sap = ChatOpenAI(proxy_model_name=model_name_SAP, proxy_client=proxy_client)
#embeddings_model = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002', proxy_client=proxy_client)
embeddings_model_own = OpenAIEmbeddings(model="text-embedding-3-small")



In [ ]:
# only for parsing the pdf reports to markdown for the first time
"""
for file in os.listdir(DATA_DIR):
    if file.endswith(".pdf"):
        try:
            print(f"Converting {file} to markdown")
            md_text = LlamaParse(
                result_type="markdown", 
                verbose=True,
                #use_vendor_multimodal_model=True,
                #vendor_multimodal_model_name="openai-gpt-4o-mini",
                #vendor_multimodal_api_key=os.getenv("OPENAI_API_KEY"),
                language="en",
                numWorkers=5).load_data(DATA_DIR + file)
            combined_md_text = "\n\n".join([doc.text for doc in md_text])
            md_file_path = DATA_DIR + file.replace(".pdf", ".md")
            print(f"Saving markdown to {md_file_path}")
            with open(md_file_path, "w", encoding="utf-8") as f:
                f.write(combined_md_text)
            print(f"Successfully converted {file}")
        except Exception as e:
            print(f"Error converting {file}: {e}")
"""

# RAG

In [5]:
#setup embeddings and chain
retriever = setup_rag_embeddings(data_dir=DATA_DIR, faiss_path="faiss_references", embeddings_model=embeddings_model_own)
rag_chain = build_rag_chain(retriever=retriever, llm=llm_sap)

Embedding documents...
amount of documents used:  43
Saved 13062 chunks to faiss_references.


e:\Repositories\graphrag-businessqa-evaluation\.conda\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


prompt input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


Map generated questions and ground_truths into evaluation dataset


In [6]:

with open(DATASET, 'r') as f:
    synthetic_data = json.load(f)

if isinstance(synthetic_data, dict) and 'responses' in synthetic_data:
    synthetic_data = synthetic_data['responses']

queries = [item.get('question', "") for item in synthetic_data]
ground_truths = [item.get('ground_truth', "") for item in synthetic_data]
contexts = [item.get('context', "") for item in synthetic_data]

answers = []
retrieved_contexts = []

for query in queries:
    answer = rag_chain.invoke(query)
    answers.append(answer)
    print("Query: ",query)
    print("Anwer: ",answer)
    retrieved_context = [doc.page_content for doc in retriever.invoke(query)]
    retrieved_contexts.append(retrieved_context)
    print("Retrieved context:",retrieved_context)


Query:  How can integrating environmental costs into a Corporate Sustainability Management System (CSMS) create a synergistic effect that aligns sustainability with profitability, enhancing both environmental conservation and financial performance across various business domains such as finance, operations, and technology?
Anwer:  Integrating environmental costs into a Corporate Sustainability Management System (CSMS) can create a synergistic effect by aligning sustainability with profitability, as it encourages the mitigation of risk and fosters stronger connections between business and society. This approach helps maximize shareholder value while benefiting all stakeholders, including employees and community members, and promoting environmental conservation. Ultimately, it enhances financial performance across various business domains such as finance, operations, and technology.
Retrieved context: ['Corporate sustainability (CS) has become a part of the business community’s vernacula

In [7]:
# populate eval dataset

evaluation_samples = []

for query, answer, retrieved_context, ground_truth in zip(queries, answers, retrieved_contexts, ground_truths):
    sample = SingleTurnSample(
        user_input=query,
        response=answer,
        reference=ground_truth,
        retrieved_contexts=retrieved_context
    )
    evaluation_samples.append(sample)

evaluation_dataset = EvaluationDataset(samples=evaluation_samples)



In [8]:
print(evaluation_dataset.samples)
print(type(evaluation_dataset.samples))
print(type(evaluation_dataset.samples[0]))
print(dir(evaluation_dataset.samples[0]))



[SingleTurnSample(user_input='How can integrating environmental costs into a Corporate Sustainability Management System (CSMS) create a synergistic effect that aligns sustainability with profitability, enhancing both environmental conservation and financial performance across various business domains such as finance, operations, and technology?', retrieved_contexts=['Corporate sustainability (CS) has become a part of the business community’s vernacular worldwide. The origin of the CS concept is mainly linked to the Brundtland report (1987), which incorporates CS through the triple bottom line of long-term economic prosperity, social equity, and environmental responsibility into a company’s management and operations (Bansal 2005). Despite the effective', 'ance but also provides the opportunity to work for the mitiga-\n\ntion of risk aversion and bridging gaps between business and\n\nsociety (Morgan et al. 2009). Furthermore, CS addresses the\n\nissues of well-being and environmental deg

In [9]:

evaluator_llm = LangchainLLMWrapper(llm_sap)
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

metrics = [
    FactualCorrectness(llm=evaluator_llm, mode="precision", name="FactualCorrectness_Precision"),
    FactualCorrectness(llm=evaluator_llm, mode="recall", name="FactualCorrectness_Recall"),
    FactualCorrectness(llm=evaluator_llm, mode="f1", name="FactualCorrectness_F1"),
    Faithfulness(llm=evaluator_llm), 
 
]
results = evaluate(dataset=evaluation_dataset, metrics=metrics)
df_rag = results.to_pandas()


Evaluating: 100%|██████████| 128/128 [02:43<00:00,  1.28s/it]


# Results

In [10]:
pd.set_option('display.max_colwidth', 200) # default is 50 / None for unlimited

df_rag

# context recall = measures how many of the relevant documents (or pieces of information) were successfully retrieved
# factual correctness (precision) = proportion of correct claims made that also found in the reference / how many are correct 
# factual correctness (recall) = proportion of facts in the reference that are also present in the response / how many are found
# faithfulness = are the claims made in the response supported by the retrieved context
# semantic similarity = how similar the response is to the ground truth
 


,user_input,retrieved_contexts,response,reference,FactualCorrectness_Precision,FactualCorrectness_Recall,FactualCorrectness_F1,faithfulness
0,"How can integrating environmental costs into a Corporate Sustainability Management System (CSMS) create a synergistic effect that aligns sustainability with profitability, enhancing both environme...","[Corporate sustainability (CS) has become a part of the business community’s vernacular worldwide. The origin of the CS concept is mainly linked to the Brundtland report (1987), which incorporates...","Integrating environmental costs into a Corporate Sustainability Management System (CSMS) can create a synergistic effect by aligning sustainability with profitability, as it encourages the mitigat...",Integrating environmental costs into a company's accounting sector through a Corporate Sustainability Management System (CSMS) not only fosters environmental conservation but also enhances overall...,0.29,0.33,0.33,0.400000
1,"How can organizations strategically integrate a Green Shared Vision to enhance psychological ownership of green products and mitigate green confusion, thereby aligning with ESG investing to optimi...","[Keywords: green shared vision; psychological ownership theory; green product psychological ownership; employee green confusion; employee environmental performance\n\n# 1. Introduction, shared vis...","Organizations can strategically integrate a Green Shared Vision (GSV) by using psychological ownership theory to foster Green Product Psychological Ownership (GPPO), which can enhance employee env...","To maximize employees' behavior and performance, it is crucial to foster a robust Green Shared Vision that enhances psychological ownership of green products, thereby mitigating green confusion an...",0.44,0.33,0.38,0.222222
2,"How can companies strategically integrate sustainable human resource management practices with comprehensive governance structures, including board diversity and size, to enhance corporate sustain...",[Socially responsible human resource management manifests itself in the treatment of employees as important stakeholders of the enterprise [54]. Diversity management will be used to study the soci...,Companies can integrate sustainable HRM practices with governance structures by treating employees as key stakeholders and aligning HR strategies with diversity management goals. This alignment ca...,"The sustainable management of human resources (SMHR) should be intricately aligned with a company's comprehensive sustainable development strategy, encompassing not only the fair treatment and wel...",0.20,0.40,0.48,0.222222
3,"How can multinational corporations in environmentally risky sectors develop a unified strategy for corporate sustainability that effectively integrates ownership structure, financial performance, ...","[Corporate sustainability requires managers to simultaneously address widely diverging but interconnected concerns for the natural environment, social welfare, and economic prosperity (Bansal, 200...","Multinational corporations can develop a unified sustainability strategy by integrating sustainability into long-term planning, strategic management, and activities, ensuring alignment with both i...","Corporate sustainability performance is intricately determined by factors such as ownership concentration, company profitability, firm size, growth opportunities, and debt, particularly in environ...",0.00,0.00,0.00,0.470588
4,"How can businesses strategically integrate corporate sustainability into their management accounting practices to enhance both financial performance and corporate social responsibility, while over...","[Organizations have been showing a growing awareness regarding the importance of corporate sustainability. However, the integration of sustainability concerns in companies' long-term planning, str...","Businesses can strategically integrate corporate sustainability into management accounting by embedding sustainability conc

In [11]:
average_metrics = df_rag[['FactualCorrectness_Precision', 'FactualCorrectness_Recall', 'FactualCorrectness_F1']].mean().round(4)

print("Average for each metric:")
print(average_metrics)

Average for each metric:
FactualCorrectness_Precision    0.3241
FactualCorrectness_Recall       0.3884
FactualCorrectness_F1           0.3731
dtype: float64


# GraphRAG

## Indexing

In [ ]:
# graphrag initial setup
!mkdir ./graphrag
!mkdir ./input_references
!python -m graphrag init --root ./graphrag

In [12]:
# indexing, only run once unless you want to update the index
!python -m graphrag index --root ./graphrag


⠋ GraphRAG Indexer 
Logging enabled at 
E:\Repositories\graphrag-businessqa-evaluation\graphrag\logs\indexing-engine.lo
g
⠋ GraphRAG Indexer 
⠋ GraphRAG Indexer 
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (45 filtered)  100%  …
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (45 filtered)  100%  …
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (45 filtered)  100%  …
└── create_base_text_units
⠴ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (45 filtered)  100%  …
└── create_base_text_units
⠇ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (45 filtered)  100%  …
└── create_base_text_units
⠋ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (45 filtered)  100%  …
└── create_base_text_units
⠧ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 43 files loaded (45 filtered)  100%  …
└─

## Query Engine

### Global Search

follows the implementations guide by the docs https://microsoft.github.io/graphrag/examples_notebooks/global_search/

In [ ]:
from graphrag_utils import setup_graphrag
import pandas as pd
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
import os
import json
import asyncio

api_key = os.environ["GRAPHRAG_API_KEY"]

llm = ChatOpenAI(
    api_key=api_key,
    model="gpt-4o-mini",
    api_type=OpenaiApiType.OpenAI,
    max_retries=3,
)
community_level = 1

with open(DATASET, 'r') as file:
    data = json.load(file)
    
queries = [response['question'] for response in data['responses']]
references = [response['ground_truth'] for response in data['responses']]

search_engine_global = setup_graphrag(model_name, llm, community_level)

async def perform_global_search(query):
    print(f"Performing search with query: {query}")
    result = await search_engine_global.asearch(query)
    print(f"Result for query: {query} is: {result.response}")
    return result.response

async def run_queries_sequentially(queries, references):
    evaluation_samples = []
    for i, (query, reference) in enumerate(zip(queries, references)):
        result = await perform_global_search(query)
        sample = SingleTurnSample(
            user_input=query,
            response=result,
            reference=reference
        )
        evaluation_samples.append(sample)

        # If this is not the last iteration, delay for 90 seconds
        if i < len(queries) - 1:
            await asyncio.sleep(90)  # 1.5 minutes delay

    return evaluation_samples

# Execute in a Jupyter cell
evaluation_samples = await run_queries_sequentially(queries, references)

evaluation_dataset = EvaluationDataset(samples=evaluation_samples)


Missing reports for communities: [43, 12, 23, 4, 11, 42, 2, 25, 14, 26, 27, 29, 18, 38, 15, 32, 1, 36, 31, 39, 22, 28, 13, 37, 30, 16, 47, 6, 44, 49, 7, 48, 24, 35, 20, 45, 50, 41, 33, 19, 40, 379, 364, 246, 80, 161, 144, 169, 362, 149, 280, 403, 365, 293, 342, 130, 184, 370, 369, 338, 124, 133, 353, 129, 366, 317, 381, 383, 314, 358, 132, 110, 304, 274, 399, 345, 230, 347, 315, 312, 350, 275, 346, 322, 325, 343, 302, 352, 348, 397, 400, 354, 367, 308, 269, 281, 290, 284, 286, 303, 331, 291, 289, 277, 320, 299, 283, 298, 297, 300, 301, 413, 382, 251, 318, 412, 282, 295, 285, 378, 296, 416, 363, 360, 387, 173, 385, 390, 388, 386, 392, 418, 407, 361, 408, 406, 270, 417, 398, 377, 404, 307, 334, 368, 309, 376, 336, 375, 356, 419, 310, 339, 344, 384, 372, 337, 401, 405, 159, 330, 335, 332, 333, 340, 311, 306, 329, 394, 328, 359, 323, 414, 409, 410, 393, 287, 411, 389, 374, 324, 326, 327, 396, 279, 355, 402, 305, 288, 415, 634, 628, 456, 765, 650, 469, 573, 776, 683, 529, 509, 884, 541, 840

                                     id  human_readable_id  community  level  \
0  eb3dcb87-2f35-41f6-acd4-ab9b2b0ddefe               1071       1071      4   
1  3c26ae78-a0fe-4aba-a560-fb6423582011               1072       1072      4   
2  81cb72d7-a386-4cf3-8d1f-c8935e6fc1cf               1073       1073      4   
3  b45e85bf-2ddc-4d70-b911-0c9eb360f9b2               1074       1074      4   
4  fdf8ff70-2451-4c40-9336-c115ca1e646f               1075       1075      4   

                                                title  \
0       Corporate Environmental Performance Community   
1            A. M. Romi and Kelley School of Business   
2  Corporate Social Responsibility Research Community   
3      Cowen et al. and Correlation Studies Community   
4            Panel B and Emissions Analysis Community   

                                                                                                                                                                                

In [15]:
from graphrag_utils import setup_graphrag
import pandas as pd
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
import os
import json
import asyncio

api_key = os.environ["GRAPHRAG_API_KEY"]


llm = ChatOpenAI(
    api_key=api_key,
    model="gpt-4o-mini",
    api_type=OpenaiApiType.OpenAI,
    max_retries=20,
)
community_level = 1

with open(DATASET, 'r') as file:
    data = json.load(file)
    
queries = [response['question'] for response in data['responses']]
references = [response['ground_truth'] for response in data['responses']]

search_engine_global = setup_graphrag(model_name, llm , community_level)

async def perform_global_search(query):
    print(f"Performing search with query: {query}")
    result = await search_engine_global.asearch(query)
    print(f"Result for query: {query} is: {result.response}")
    return result.response

tasks = [perform_global_search(query) for query in queries]
results = await asyncio.gather(*tasks)

evaluation_samples = []
for query, result, reference in zip(queries, results, references):
    sample = SingleTurnSample(
        user_input=query,
        response=result,
        reference=reference
    )
    evaluation_samples.append(sample)

evaluation_dataset = EvaluationDataset(samples=evaluation_samples)
#print(evaluation_dataset)

Missing reports for communities: [43, 12, 23, 4, 11, 42, 2, 25, 14, 26, 27, 29, 18, 38, 15, 32, 1, 36, 31, 39, 22, 28, 13, 37, 30, 16, 47, 6, 44, 49, 7, 48, 24, 35, 20, 45, 50, 41, 33, 19, 40, 379, 364, 246, 80, 161, 144, 169, 362, 149, 280, 403, 365, 293, 342, 130, 184, 370, 369, 338, 124, 133, 353, 129, 366, 317, 381, 383, 314, 358, 132, 110, 304, 274, 399, 345, 230, 347, 315, 312, 350, 275, 346, 322, 325, 343, 302, 352, 348, 397, 400, 354, 367, 308, 269, 281, 290, 284, 286, 303, 331, 291, 289, 277, 320, 299, 283, 298, 297, 300, 301, 413, 382, 251, 318, 412, 282, 295, 285, 378, 296, 416, 363, 360, 387, 173, 385, 390, 388, 386, 392, 418, 407, 361, 408, 406, 270, 417, 398, 377, 404, 307, 334, 368, 309, 376, 336, 375, 356, 419, 310, 339, 344, 384, 372, 337, 401, 405, 159, 330, 335, 332, 333, 340, 311, 306, 329, 394, 328, 359, 323, 414, 409, 410, 393, 287, 411, 389, 374, 324, 326, 327, 396, 279, 355, 402, 305, 288, 415, 634, 628, 456, 765, 650, 469, 573, 776, 683, 529, 509, 884, 541, 840

                                     id  human_readable_id  community  level  \
0  eb3dcb87-2f35-41f6-acd4-ab9b2b0ddefe               1071       1071      4   
1  3c26ae78-a0fe-4aba-a560-fb6423582011               1072       1072      4   
2  81cb72d7-a386-4cf3-8d1f-c8935e6fc1cf               1073       1073      4   
3  b45e85bf-2ddc-4d70-b911-0c9eb360f9b2               1074       1074      4   
4  fdf8ff70-2451-4c40-9336-c115ca1e646f               1075       1075      4   

                                                title  \
0       Corporate Environmental Performance Community   
1            A. M. Romi and Kelley School of Business   
2  Corporate Social Responsibility Research Community   
3      Cowen et al. and Correlation Studies Community   
4            Panel B and Emissions Analysis Community   

                                                                                                                                                                                

CancelledError: 

In [13]:
print(evaluation_dataset)


EvaluationDataset(features=['user_input', 'response', 'reference'], len=5)


In [14]:
evaluator_llm = LangchainLLMWrapper(llm_sap)
#evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

metrics = [
    FactualCorrectness(llm=evaluator_llm, mode="precision", name="FactualCorrectness_Precision"),
    FactualCorrectness(llm=evaluator_llm, mode="recall", name="FactualCorrectness_Recall"),
    FactualCorrectness(llm=evaluator_llm, mode="f1", name="FactualCorrectness_F1"),
    #Faithfulness(llm=evaluator_llm), 
]
results = evaluate(dataset=evaluation_dataset, metrics=metrics)
df_grag = results.to_pandas()



Evaluating: 100%|██████████| 15/15 [01:25<00:00,  5.73s/it]


In [15]:
pd.set_option('display.max_colwidth', 500) # default is 50 / None for unlimited
df_grag

,user_input,response,reference,FactualCorrectness_Precision,FactualCorrectness_Recall,FactualCorrectness_F1
0,"How can a company strategically integrate a Corporate Sustainability Management System (CSMS) with a Green Shared Vision and a comprehensive evaluation framework to simultaneously enhance environmental conservation, improve financial performance, and foster stronger stakeholder relationships, while addressing specific challenges such as green confusion, social sustainability, and the impact of external factors like climate vulnerability and pandemics?","## Strategic Integration of CSMS with Green Shared Vision\n\nTo effectively integrate a Corporate Sustainability Management System (CSMS) with a Green Shared Vision (GSV), companies must ensure that their sustainability goals are aligned with their core business strategies. This alignment is crucial as it embeds sustainability into the organizational culture, leading to improved environmental conservation efforts and enhanced financial performance. A CSMS provides a structured approach that ...","Endiana et al. assert that the accounting sector of a company can foster environmental conservation and enhance performance through the implementation of a Corporate Sustainability Management System (CSMS), by appropriately allocating environmental costs, thereby improving financial outcomes and customer loyalty. From the employees' perspective, Chang et al. emphasize the importance of establishing a solid Green Shared Vision to mitigate green confusion and boost economic performance, highli...",0.08,0.45,0.33
1,"How can multinational corporations effectively integrate sustainable human resource management practices with innovative environmental strategies and diverse team dynamics to enhance their overall corporate sustainability performance, while simultaneously addressing socio-environmental, socio-economical, and eco-efficiency dimensions across different global markets?","# Integrating Sustainable HRM with Environmental Strategies and Diversity\n\nMultinational corporations (MNCs) face the challenge of integrating sustainable human resource management (HRM) practices with innovative environmental strategies and diverse team dynamics to enhance their overall corporate sustainability performance. This integration is essential for addressing the socio-environmental, socio-economical, and eco-efficiency dimensions across different global markets.\n\n## Sustainabl...","Mazur and Walczynia, in alignment with Chang et al., emphasize the importance of implementing sustainable management of human resources (SMHR) when a company has a robust corporate sustainability perspective, ensuring that current needs are met without jeopardizing future stakeholders' needs. Complementing this, Xia et al. identify practices that enhance corporate sustainability performance across socio-environmental, socio-economical, and eco-efficiency dimensions by fostering environmental...",0.23,0.25,0.30
2,"How can multinational corporations effectively integrate Global Reporting Initiative (GRI) indicators to achieve corporate sustainability, considering the diverse interpretations of sustainability across different countries, while also leveraging strategic management frameworks to balance the creation of value for business, society, and nature in their operations and product offerings?",# Integrating GRI Indicators for Corporate Sustainability\n\nMultinational corporations (MNCs) face the challenge of effectively integrating Global Reporting Initiative (GRI) indicators into their sustainability strategies while navigating the diverse interpretations of sustainability across different cultural and regulatory contexts. This integration is essential for aligning corporate practices with sustainability goals and enhancing accountability and transparency in reporting.\n\n## Unde...,"In the context of growing global concern for companies to align their corporate sustainability goals with international standar

In [16]:
average_metrics_grag = df_grag[['FactualCorrectness_Precision', 'FactualCorrectness_Recall', 'FactualCorrectness_F1']].mean().round(4)
average_metrics_rag = df_rag[['FactualCorrectness_Precision', 'FactualCorrectness_Recall', 'FactualCorrectness_F1']].mean().round(4)

print("Average for each metric (GraphRAG):")
print(average_metrics_grag)
print("Average for each metric (RAG):")
print(average_metrics_rag)


Average for each metric (GraphRAG):
FactualCorrectness_Precision    0.264
FactualCorrectness_Recall       0.472
FactualCorrectness_F1           0.334
dtype: float64
Average for each metric (RAG):
FactualCorrectness_Precision    0.412
FactualCorrectness_Recall       0.262
FactualCorrectness_F1           0.322
dtype: float64
